# Loading Data:

# Importing libraries

In [1]:
# Importing the necessary Libraries
import pandas as pd #Data manipulation
import numpy as np #To deal with columns
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

import matplotlib.pyplot as plt #visualization
import seaborn as sns #advanced plot
sns.set()

from datetime import timedelta #In RFM analysis, timedelta is used to calculate Recency by measuring the difference between a customer's last purchase date and a reference date.

import warnings
warnings.filterwarnings('ignore')

# Generating data

In [2]:
# Generate sample data
n_rows = 30000
user_ids = [f'{np.random.randint(10000, 99999)}-CNTR-C' for _ in range(n_rows)]
datetimes = pd.to_datetime('2023-01-01') + pd.to_timedelta(np.random.randint(0, 365, n_rows), unit='D')
source_case_counts = np.random.randint(1, 10, n_rows)
total_prices = np.random.uniform(10, 1000, n_rows)

# Create the DataFrame
df = {'User_id': user_ids,
        'DateTime': datetimes,
        'Count of SOURCE_CASE_ID': source_case_counts,
        'Total Price': total_prices}
data = pd.DataFrame(df)

# Display the first few rows
data.head()


,User_id,DateTime,Count of SOURCE_CASE_ID,Total Price
0,46745-CNTR-C,2023-01-05,4,66.832803
1,11062-CNTR-C,2023-12-23,7,227.800019
2,70183-CNTR-C,2023-08-08,8,735.753574
3,44813-CNTR-C,2023-01-09,6,863.027050
4,39908-CNTR-C,2023-12-27,3,513.953635


# Explore the data

In [3]:
# Display data types and non-null counts
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   User_id                  30000 non-null  object        
 1   DateTime                 30000 non-null  datetime64[ns]
 2   Count of SOURCE_CASE_ID  30000 non-null  int64         
 3   Total Price              30000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 937.6+ KB
None


In [4]:
unique_user_count = data['User_id'].nunique() # Access the 'User_id' column from the DataFrame 'df'
print(f"The unique count of User_id is: {unique_user_count}")

The unique count of User_id is: 25576


# Outlier

In [5]:
#just remove 99th and 100 th percentle of outlier in the monetary Total Price

# Calculate the 99th and 100th percentiles
percentile_99 = data['Total Price'].quantile(0.99)
percentile_100 = data['Total Price'].quantile(1.00)

# Filter out outliers
data = data[(data['Total Price'] <= percentile_99)]

# RFM Modelling

To do the RFM analysis, we need to create 3 features from the data:

**Recency** - Latest date - Last invoice date. (Number of days since the last purchase date)

**Frequency** - count of invoice numbers. (Total number of transactions made by a unique customer)

**Monetary** - Sum of Total sales. (Total value of transacted sales by each customer)

Now, let's create a function which can be used to generate the RFM features.



In [6]:
def RFM_Features(df, customerID, invoiceDate, transID, sales):
    ''' Create the Recency, Frequency, and Monetary features from the data '''
    # Final date in the data + 1 to create latest date
    latest_date = df[invoiceDate].max() + timedelta(1)
    # RFM feature creation
    RFMScores = df.groupby(customerID).agg({invoiceDate: lambda x: (latest_date - x.max()).days,
                                          transID: lambda x: len(x),
                                          sales: lambda x: sum(x)})
    # Converting invoiceDate to int since this contains number of days
    RFMScores[invoiceDate] = RFMScores[invoiceDate].astype(int)
    # Renaming column names to Recency, Frequency and Monetary
    RFMScores.rename(columns={invoiceDate: 'Recency',
                              transID: 'Frequency',
                              sales: 'Monetary'}, inplace=True)
    return RFMScores.reset_index()

RFM = RFM_Features(df=data, customerID= "User_id", invoiceDate = "DateTime", transID= "Count of SOURCE_CASE_ID", sales="Total Price")
print(RFM.shape)
RFM.head()

(25362, 4)


,User_id,Recency,Frequency,Monetary
0,10005-CNTR-C,151,1,769.521391
1,10006-CNTR-C,347,1,733.860955
2,10007-CNTR-C,277,1,984.597111
3,10008-CNTR-C,8,3,1291.359262
4,10014-CNTR-C,279,1,167.840174


To conduct RFM analysis, we need to rank the customers based on each RFM attribute separately.

Assume that we rank these customers from 1-5 using RFM values. (1-low score & 5-High score)

**Steps to be followed for RFM ranking:**
1. Sort the Recency column by **most recent purchases at the top.** For Frequency and Monetary features sort it by the **most frequent** and **most valuable** purchases at the top.
2. If you are using N-scale ranking to rank the customers then you have to divide the sorted values of the features into 1/N groups. Here, we are using 4-scale ranking hence we need to divide the values into 4 groups.

we can do both the sorting and grouping using pandas **df.quantile** method by providing the number of quantiles as a list.





**NOTE:** The value of N decides the number of different RFM rank groups you want to create. All possible combination of ranks from 1-N for all the three RFM features results in N^3 rank groups ranging from 111(lowest) to NNN(highest).

In our case N=4, hence we could have a maximum of 4^3 = 64 rank groups with scores from 111 to 444.

In [7]:
# Creating quantiles
Quantiles = RFM[['Recency', 'Frequency', 'Monetary']].quantile([0.25, 0.50, 0.75])
Quantiles = Quantiles.to_dict()
Quantiles

{'Recency': {0.25: 82.0, 0.5: 169.0, 0.75: 264.0},
 'Frequency': {0.25: 1.0, 0.5: 1.0, 0.75: 1.0},
 'Monetary': {0.25: 298.1617374617638,
  0.5: 569.9819299397838,
  0.75: 825.3052077532241}}

In [8]:
# Creating RFM ranks
def RFMRanking(x, variable, quantile_dict):
    ''' Ranking the Recency, Frequency, and Monetary features based on quantile values '''

    # checking if the feature to rank is Recency
    if variable == 'Recency':
        if x <= quantile_dict[variable][0.25]:
            return 4
        elif (x > quantile_dict[variable][0.25]) & (x <= quantile_dict[variable][0.5]):
            return 3
        elif (x > quantile_dict[variable][0.5]) & (x <= quantile_dict[variable][0.75]):
            return 2
        else:
            return 1

    # checking if the feature to rank is Frequency and Monetary
    if variable in ('Frequency','Monetary'):
        if x <= quantile_dict[variable][0.25]:
            return 1
        elif (x > quantile_dict[variable][0.25]) & (x <= quantile_dict[variable][0.5]):
            return 2
        elif (x > quantile_dict[variable][0.5]) & (x <= quantile_dict[variable][0.75]):
            return 3
        else:
            return 4

In [9]:
RFM['R'] = RFM['Recency'].apply(lambda x: RFMRanking(x, variable='Recency', quantile_dict=Quantiles))
RFM['F'] = RFM['Frequency'].apply(lambda x: RFMRanking(x, variable='Frequency', quantile_dict=Quantiles))
RFM['M'] = RFM['Monetary'].apply(lambda x: RFMRanking(x, variable='Monetary', quantile_dict=Quantiles))

In [10]:
print(RFM.shape)
RFM.head()

(25362, 7)


,User_id,Recency,Frequency,Monetary,R,F,M
0,10005-CNTR-C,151,1,769.521391,3,1,3
1,10006-CNTR-C,347,1,733.860955,1,1,3
2,10007-CNTR-C,277,1,984.597111,1,1,4
3,10008-CNTR-C,8,3,1291.359262,4,4,4
4,10014-CNTR-C,279,1,167.840174,1,1,1


In [12]:
RFM['Group'] = RFM['R'].apply(str) + RFM['F'].apply(str) + RFM['M'].apply(str)
RFM.head()

,User_id,Recency,Frequency,Monetary,R,F,M,Score,Group
0,10005-CNTR-C,151,1,769.521391,3,1,3,7,313
1,10006-CNTR-C,347,1,733.860955,1,1,3,5,113
2,10007-CNTR-C,277,1,984.597111,1,1,4,6,114
3,10008-CNTR-C,8,3,1291.359262,4,4,4,12,444
4,10014-CNTR-C,279,1,167.840174,1,1,1,3,111


In [11]:
RFM["Score"] = RFM[['R', 'F', 'M']].sum(axis=1)
RFM.head()

,User_id,Recency,Frequency,Monetary,R,F,M,Score
0,10005-CNTR-C,151,1,769.521391,3,1,3,7
1,10006-CNTR-C,347,1,733.860955,1,1,3,5
2,10007-CNTR-C,277,1,984.597111,1,1,4,6
3,10008-CNTR-C,8,3,1291.359262,4,4,4,12
4,10014-CNTR-C,279,1,167.840174,1,1,1,3


In [13]:
# Loyalty levels
loyalty = ['Lost Customers','Occasional Customers', 'New Customers', 'At-Risk Customers', 'Loyal Customers','High-Value Customers']
RFM['Loyalty_Level'] = pd.qcut(RFM['Score'], q=6, labels= loyalty)
RFM.head()

,User_id,Recency,Frequency,Monetary,R,F,M,Score,Group,Loyalty_Level
0,10005-CNTR-C,151,1,769.521391,3,1,3,7,313,At-Risk Customers
1,10006-CNTR-C,347,1,733.860955,1,1,3,5,113,Occasional Customers
2,10007-CNTR-C,277,1,984.597111,1,1,4,6,114,New Customers
3,10008-CNTR-C,8,3,1291.359262,4,4,4,12,444,High-Value Customers
4,10014-CNTR-C,279,1,167.840174,1,1,1,3,111,Lost Customers


In [ ]:
behaviour = RFM.groupby('Loyalty_Level')[['Recency', 'Frequency', 'Monetary', 'Score']].mean()
behaviour

,Recency,Frequency,Monetary,Score
Loyalty_Level,,,,
Lost Customers,283.678336,1.000000,249.381251,3.660332
Occasional Customers,221.078640,1.000000,431.253010,5.000000
New Customers,163.647258,1.001696,516.940711,6.000000
At-Risk Customers,122.092486,1.013763,655.577888,7.000000
Loyal Customers,90.087150,1.185956,793.665685,8.370247
High-Value Customers,96.540299,2.142083,1157.474261,11.082510


In [14]:
# Group by User_id and get the latest DateTime
latest_data = data.groupby('User_id')['DateTime'].max().reset_index()

# Merge with RFM table on User_id
RFM = pd.merge(RFM, latest_data, on='User_id', how='left')

# Display the updated RFM table
RFM.head()

,User_id,Recency,Frequency,Monetary,R,F,M,Score,Group,Loyalty_Level,DateTime
0,10005-CNTR-C,151,1,769.521391,3,1,3,7,313,At-Risk Customers,2023-08-03
1,10006-CNTR-C,347,1,733.860955,1,1,3,5,113,Occasional Customers,2023-01-19
2,10007-CNTR-C,277,1,984.597111,1,1,4,6,114,New Customers,2023-03-30
3,10008-CNTR-C,8,3,1291.359262,4,4,4,12,444,High-Value Customers,2023-12-24
4,10014-CNTR-C,279,1,167.840174,1,1,1,3,111,Lost Customers,2023-03-28


In [17]:

from datetime import datetime, timedelta

# Define the threshold date (6 months before the current date)
current_date = datetime.now()
six_months_ago = current_date - timedelta(days=180)

# Categorize users
RFM["6_Month_Category"] = RFM["DateTime"].apply(lambda x: "Last 6 Months" if x >= six_months_ago else "Older than 6 Months")

In [18]:
RFM.head()

,User_id,Recency,Frequency,Monetary,R,F,M,Score,Group,Loyalty_Level,DateTime,6_Month_Category
0,10005-CNTR-C,151,1,769.521391,3,1,3,7,313,At-Risk Customers,2023-08-03,Older than 6 Months
1,10006-CNTR-C,347,1,733.860955,1,1,3,5,113,Occasional Customers,2023-01-19,Older than 6 Months
2,10007-CNTR-C,277,1,984.597111,1,1,4,6,114,New Customers,2023-03-30,Older than 6 Months
3,10008-CNTR-C,8,3,1291.359262,4,4,4,12,444,High-Value Customers,2023-12-24,Older than 6 Months
4,10014-CNTR-C,279,1,167.840174,1,1,1,3,111,Lost Customers,2023-03-28,Older than 6 Months


In [19]:
# prompt: export RFM into excel sheet

# Export the RFM DataFrame to an Excel file
RFM.to_excel('RFM_Analysis.xlsx', index=False)

# Download the Excel file from Colab
from google.colab import files
files.download('RFM_Analysis.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
behaviour = RFM.groupby(['Loyalty_Level', '6_Month_Category'])[['Recency', 'Frequency', 'Monetary', 'Score']].mean()

In [21]:
behaviour

,,Recency,Frequency,Monetary,Score
Loyalty_Level,6_Month_Category,,,,
Lost Customers,Older than 6 Months,283.231872,1.000000,244.005284,3.643304
Occasional Customers,Older than 6 Months,221.765725,1.000000,433.178804,5.000000
New Customers,Older than 6 Months,169.158954,1.001129,534.123978,6.000000
At-Risk Customers,Older than 6 Months,119.660603,1.017649,647.188261,7.000000
Loyal Customers,Older than 6 Months,93.034078,1.192737,796.919190,8.367598
High-Value Customers,Older than 6 Months,94.695377,2.132679,1146.845027,11.069348


In [22]:
# prompt: Loyalty_Level	6_Month_Category include the count
loyalty_level_counts = RFM.groupby(['Loyalty_Level', '6_Month_Category']).size().reset_index(name='Count')
loyalty_level_counts

,Loyalty_Level,6_Month_Category,Count
0,Lost Customers,Older than 6 Months,4951
1,Occasional Customers,Older than 6 Months,4674
2,New Customers,Older than 6 Months,5316
3,At-Risk Customers,Older than 6 Months,3683
4,Loyal Customers,Older than 6 Months,3580
5,High-Value Customers,Older than 6 Months,3158


In [23]:
import pandas as pd
# Calculate the average score for each group
average_scores = RFM.groupby(['Loyalty_Level', '6_Month_Category'])['Score'].mean().reset_index(name='Average_Score')

# Merge the average scores with the loyalty_level_counts DataFrame
loyalty_level_counts = pd.merge(loyalty_level_counts, average_scores, on=['Loyalty_Level', '6_Month_Category'], how='left')

loyalty_level_counts

,Loyalty_Level,6_Month_Category,Count,Average_Score
0,Lost Customers,Older than 6 Months,4951,3.643304
1,Occasional Customers,Older than 6 Months,4674,5.000000
2,New Customers,Older than 6 Months,5316,6.000000
3,At-Risk Customers,Older than 6 Months,3683,7.000000
4,Loyal Customers,Older than 6 Months,3580,8.367598
5,High-Value Customers,Older than 6 Months,3158,11.069348


# Data visualization for insights

In [24]:
import plotly.express as px

corr_matrix = RFM[['Recency', 'Frequency', 'Monetary', "Group", "Score"]].corr()

fig = px.imshow(corr_matrix, text_auto=True, color_continuous_scale="RdBu_r")
fig.update_layout(title="Correlation Matrix of RFM Features")
fig.show()

In [25]:
import plotly.express as px

loyalty_counts = RFM["Loyalty_Level"].value_counts()
fig = px.pie(values=loyalty_counts.values, names=loyalty_counts.index, title="Percentage of Customers in Each Loyalty Level", color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_traces(textinfo='percent+label')
fig.show()

In [26]:
import plotly.express as px

# Plot the histogram using Plotly
fig = px.histogram(RFM, x="Loyalty_Level", title="Distribution of Loyalty Levels",
                   labels={"Loyalty_Level": "Loyalty Level", "count": "Number of Customers"})
fig.show()

In [27]:

# Plot the histogram for RFM_Score using Plotly
fig = px.histogram(RFM, x="Score", nbins=15,
                   title='Distribution of RFM Scores',
                   labels={"RFM_Score": "Score", "count": "Number of Customers"},
                   marginal="box", # Add a box plot to show distribution
                   color_discrete_sequence=["skyblue"]) # Customize color

fig.update_layout(
    xaxis_title_text='Score',  # Customize x-axis label
    yaxis_title_text='Number of Customers',  # Customize y-axis label
    bargap=0.1, # Adjust gap between bars
)


fig.show()

In [28]:
import plotly.express as px

fig = px.scatter(RFM, x="Recency", y="Frequency", color="Loyalty_Level",
                 color_continuous_scale="coolwarm", size_max=100)
fig.update_layout(title="Frequency vs. Recency by Loyalty Level",
                  xaxis_title="Recency",
                  yaxis_title="Frequency",
                  legend_title="Loyalty Level")
fig.show()

In [29]:
# prompt: plot whisker plot to identify outlier for monetary  using ploty

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Box(y=RFM['Monetary'], name='Monetary Value'))
fig.update_layout(title='Monetary Value Box Plot', yaxis_title='Monetary Value')
fig.show()


In [31]:
import plotly.express as px

fig = px.scatter(RFM, x="Recency", y="Monetary", color="Loyalty_Level",
                 color_continuous_scale="coolwarm", size_max=100)
fig.update_layout(title="Monetary vs. Recency by Loyalty Level",
                  xaxis_title="Recency",
                  yaxis_title="Monetary",
                  legend_title="Loyalty Level")
fig.show()

## How RFM features helps in identifying the propensity to purchase of users?

There are two broad ways in which the RFM features can be utilized:
* **Preferrential Treatments:** RFM features and loyalty tags will be available only for existing users. So, whenever old users shows interest in any product, based on their loyalty tags we can target them with personalized discounts etc.


* **High value path:** We can analyze the behaviour exhibited by the old users and create multiple behaviour paths that leads to increased purchase. This learning can then be used to nudge new users to follow such paths and hence increase the probability of purchase.

